In [ ]:
# Indução de regras
"""
    Entendendo como que o aprendizado de regras funciona:
    
    Quando pegamos uma arvore de regras, o algoritmo vai fazer caminhos até obter aquele valor, ja no caso do do naïve bayes, ele vai fazer uma verificação estatistica das condições
    e vai selecionar a mais provavel, de acordo com os levantamentos feitos da base de dados
    
    Ja na apprendizagem por regras, o algoritmo vai criar regras fixas para obter as melhores condições.
    
"""
import pickle
import pandas as pd

with open('./zegodata.pkl', "rb") as data:
    previsores, classe = pickle.load(data)

with open('./zegotestdata.pkl', 'rb') as data:
    previsores_teste, classe_teste = pickle.load(data)


# Testando valores e verificando o tamanho das bases coletadas e mantidas no pickle

# print(previsores, classe)
# print(previsores_teste, classe_teste)
# print(len(previsores), len(previsores_teste))

# Algoritmo OpenR(ules)
<h3>Neste tipo a gente precisa testar antes com uma quantidade menor de dados para testar as regras que queremos aplicar</h3>


Explicando e exemplificando a história

nos fazemos uma tabela, onde a primeira coluna é o atributo, a segunda coluna é a classe que queremos verificar, a terceira coluna é a regra, a quarta coluna é o erro e a quinta é o erro somado, então:


<!-- Lembrar de fazer uma tabela de verdade aqui, nao deixar simplesmente alinhado pq nao vai adiantar muito -->
|Atributo     |quantidade         |regras                             |erro      |total erro|
|--- |--- |--- |--- |--- |
|A            |ruim = 10          |se ruim, entao alto                 |1/5         |       |
|b            |desconhecida = 3   |se desconhecida entao moderado      |3/5         |6/15   |
|c            |boa = 1            |se boa entao baixo                  |2/5         |        |

# Algoritmo PRISM

No caso do prism, nos pegamos apenas uma condição, e dela nos vamos pegar a maior abrangencia e determinar qual vai ser a melhor condiaçao para o nosso algoritmo


